In [173]:
import spacy
from spacy import displacy
#source .venv/bin/activate

In [174]:
zlp = spacy.load('zh_core_web_lg')
elp = spacy.load('en_core_web_lg')


sentence1 = "规划一经批复，任何部门和个人不得随意修改、违规变更，防止出现换一届党委和政府改一次规划"#"众议院监督与政府改革委员会主席、来自肯塔基州的共和党人詹姆斯 于今年1月启动了相关调查"# 
sentence2 = "The plan, once criticized, must not be modified or altered by any department or individual to prevent another Party Committee or Government from changing the plan."
#"When I came home I was tired"
# Parse the sentence using spaCy
doc1 = zlp(sentence1)
doc2 = elp(sentence2)

# Visualize the dependency tree with default settings
displacy.render(doc1, style='dep', jupyter=True)
displacy.render(doc2, style='dep', jupyter=True)

print("doc1:")
for token in doc1:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

print("doc2:")
for token in doc2:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

doc1:
规划 nsubj 批复 VERB []
一经 advmod 批复 VERB []
批复 dep 修改 VERB [规划, 一经]
， punct 修改 VERB []
任何 det 部门 NOUN []
部门 conj 个人 NOUN [任何]
和 cc 个人 NOUN []
个人 nsubj 修改 VERB [部门, 和]
不得 advmod 修改 VERB []
随意 advmod 修改 VERB []
修改 ROOT 修改 VERB [批复, ，, 个人, 不得, 随意, 、, 变更, ，, 防止]
、 punct 修改 VERB []
违规 advmod 变更 VERB []
变更 conj 修改 VERB [违规]
， punct 修改 VERB []
防止 conj 修改 VERB [出现, 换]
出现 ccomp 防止 VERB []
换 ccomp 防止 VERB [政府, 改]
一 nummod 政府 NOUN [届]
届 mark:clf 一 NUM []
党委 conj 政府 NOUN []
和 cc 政府 NOUN []
政府 dobj 换 VERB [一, 党委, 和]
改 conj 换 VERB [规划]
一 nummod 规划 NOUN [次]
次 mark:clf 一 NUM []
规划 dobj 改 VERB [一]
doc2:
The det plan NOUN []
plan nsubjpass modified VERB [The, ,, criticized, ,]
, punct plan NOUN []
once advmod criticized VERB []
criticized acl plan NOUN [once]
, punct plan NOUN []
must aux modified VERB []
not neg modified VERB []
be auxpass modified VERB []
modified ROOT modified VERB [plan, must, not, be, or, altered, .]
or cc modified VERB []
altered conj modified VERB [by, prevent]
by agent altere

In [175]:
import spacy
import nltk
from nltk.grammar import CFG
from nltk.parse import RecursiveDescentParser

def get_root(doc):
    curr = doc[0]
    while curr.head != curr:
        curr = curr.head
    return curr


def to_nltk_tree(node):
    if len(list(node.children)) > 0:
        return nltk.Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

nltk_tree1 = to_nltk_tree(get_root(doc1))
nltk_tree2 = to_nltk_tree(get_root(doc2))
nltk_tree1.pretty_print()
nltk_tree2.pretty_print()

                                 修改                                    
  _______________________________|_______________________               
 |   |   |   |   |       |           |       |           防止            
 |   |   |   |   |       |           |       |    _______|___           
 |   |   |   |   |       |           |       |   |           换         
 |   |   |   |   |       |           |       |   |        ___|_______   
 |   |   |   |   |       |           |       |   |       |           改 
 |   |   |   |   |       |           |       |   |       |           |  
 |   |   |   |   |       |           个人      |   |       政府          规划
 |   |   |   |   |       |        ___|___    |   |    ___|_______    |  
 |   |   |   |   |       批复      |       部门  变更  |   |   |       一   一 
 |   |   |   |   |    ___|___    |       |   |   |   |   |       |   |  
 ，   不得  随意  、   ，   规划      一经  和       任何  违规  出现  党委  和       届   次 

                                   modified              

In [200]:
def jaccard(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
     
    return intersection / union if union else 0

In [87]:
from itertools import permutations

arr = [[1, 2, 3], [4, 5, 6], [9, 1, 3]]

a = [1, 2, 2]
res = list(permutations(a))
ls = list(res[0])
[arr[i][l] for i in range(len(arr)) for l in ls]



[2, 3, 3, 5, 6, 6, 1, 3, 3]

In [188]:
import zss

class Node:
    def __init__(self, label, children=None):
        self.label = label
        self.children = children if children else []

    def get_children(node):
        return node.children

    def get_label(node):
        return node.label
    
    def complex_distance(token1, token2):
        return 0 if set(token1.children) == set(token2.children) else (len(token1.children))

def doc_to_tree(root):
    children = list(root.children)
    if len(children) == 0:
        return Node(root.dep_, [])
    else:
        return Node(root.dep_, [doc_to_tree(child) for child in children])
    
def mask(tree):
    tree.label = ['::']
    children = tree.get_children()
    if len(children) > 0:
        for child in children: mask(child)
    return tree

def pronoun_mask(tree):
    if tree.label != 'nsubj':
        tree.label = 0
    else:
        tree.label = 1
    children = tree.get_children()
    if len(children) > 0:
        for child in children: child = pronoun_mask(child)
    return tree

def collapse_pronouns(root):
    children = root.get_children()
    leaf_sum = 0
    branches = []
    for child in children:
        if not child.get_children():
            leaf_sum += child.get_label()
        else:
            branches.append(collapse_pronouns(child))
    return Node(root.get_label(), [Node(leaf_sum)]+branches) if root.get_children() else Node(root.get_label())

'''def unordered_distance(trees):
    dist = 0

    if len([[child.get_label() for child in tree.get_children()] for tree in trees]) == 0:
        if set([tree.get_label() for tree in trees]).size() == 0:
            return 0
        else: return 1

    for i in range(1, len(trees)): 
        children1 = trees[i-1].get_children()
        children2 = trees[i].get_children()
        labels1 = [child.get_label() for child in children1]
        labels2 = [child.get_label() for child in children2]

        if len(children1) == len(children2) and set(labels1) == set(labels2):
            print(labels1, labels2)
            n = len(children1)

            arr = [[0 for i in range(n)] for j in range(n)]
            for i in range(n):
                for j in range(n):
                    arr[i][j] = unordered_distance([children1[i], children2[j]])

            print(arr)
            perms = list(permutations([i for i in range(n)]))

            d = float("inf")
            for perm in perms:
                ps = list(perm)
                total = sum([arr[i][ps[i]] for i in range(n)])
                if total < d:
                    d = total

            dist += d

        else:
            dist+=(sum(labels1)-sum(labels2))

    return dist'''

'''tree1 = Node(1, [Node(0, [Node(1), Node(1)]), Node(0)])
tree2 = Node(1, [Node(0, [Node(1)]), Node(1)])

distance = zss.simple_distance(tree1, tree2, Node.get_children, Node.get_label)
print(f"Tree edit distance: {distance}")'''

'tree1 = Node(1, [Node(0, [Node(1), Node(1)]), Node(0)])\ntree2 = Node(1, [Node(0, [Node(1)]), Node(1)])\n\ndistance = zss.simple_distance(tree1, tree2, Node.get_children, Node.get_label)\nprint(f"Tree edit distance: {distance}")'

In [210]:
def tree_to_set(root):

    def helper(root, level):
        level = level
        elems = []
        children = root.get_children()
        labels = [child.get_label() for child in children]
        for child in children:
            elems.append((level, root.get_label(), child.get_label()))
            elems += helper(child, level + 1)
        return elems
    
    return [(0, -1, root.get_label())] + helper(root, 1)

#tree1 = Node(0, [Node(1), Node(0), Node(0, [Node(0), Node(1), Node(0)])])

In [170]:
def collapse_pronouns(root):
    children = root.get_children()
    leaf_sum = 0
    branches = []
    for child in children:
        if not child.get_children():
            leaf_sum += child.get_label()
        else:
            branches.append(collapse_pronouns(child))
    return Node(root.get_label(), [Node(leaf_sum)]+branches) if root.get_children() else Node(root.get_label())

tree1 = Node(0, [Node(1), Node(0), Node(0, [Node(0), Node(1), Node(0)])])
tree1_col = collapse_pronouns(tree1)
tree_to_set(tree1_col)

[(0, [-1, 0]), (1, [0, 1]), (1, [0, 0]), (2, [0, 1])]

In [211]:
tr1 = doc_to_tree(get_root(doc1))
#tr2 = collapse_pronouns(pronoun_mask(doc_to_tree(get_root(doc2))))
tree_to_set(collapse_pronouns(pronoun_mask(tr1)))

[(0, -1, 0),
 (1, 0, 0),
 (1, 0, 0),
 (2, 0, 1),
 (1, 0, 1),
 (2, 1, 0),
 (2, 1, 0),
 (3, 0, 0),
 (1, 0, 0),
 (2, 0, 0),
 (1, 0, 0),
 (2, 0, 0),
 (2, 0, 0),
 (3, 0, 0),
 (3, 0, 0),
 (4, 0, 0),
 (4, 0, 0),
 (5, 0, 0),
 (3, 0, 0),
 (4, 0, 0),
 (4, 0, 0),
 (5, 0, 0),
 (5, 0, 0),
 (6, 0, 0)]

In [212]:
def compare(doc1, doc2):
    set1 = set()
    set2 = set()
    ls1 = tree_to_set(collapse_pronouns(pronoun_mask(doc_to_tree(get_root(doc1)))))
    ls2 = tree_to_set(collapse_pronouns(pronoun_mask(doc_to_tree(get_root(doc2)))))
    print(ls1)
    for el in ls1:
        set1.add(el)
    for el in ls2:
        set2.add(el)
    return jaccard(set1, set2)

In [213]:
compare(doc1, doc2)

[(0, -1, 0), (1, 0, 0), (1, 0, 0), (2, 0, 1), (1, 0, 1), (2, 1, 0), (2, 1, 0), (3, 0, 0), (1, 0, 0), (2, 0, 0), (1, 0, 0), (2, 0, 0), (2, 0, 0), (3, 0, 0), (3, 0, 0), (4, 0, 0), (4, 0, 0), (5, 0, 0), (3, 0, 0), (4, 0, 0), (4, 0, 0), (5, 0, 0), (5, 0, 0), (6, 0, 0)]


0.7

In [63]:
set1 = set()
set2 = set()
set1.add('a')
set1.add('b')
set2.add('b')
set2.add('a')
print(set1, set2)
print(set1 == set2)
print(Node('g') == Node('g'))

{'b', 'a'} {'b', 'a'}
True
False


In [28]:
print(tr1.children)

[<__main__.Node object at 0x3081250d0>, <__main__.Node object at 0x308126de0>, <__main__.Node object at 0x141308c50>]
